In [26]:
import json
import lightgbm as lgb
import numpy as np
import pandas as pd

from glob import glob
from tqdm import tqdm, tqdm_notebook

from utils import FEATS_EXCLUDED, COLS_TEST1, COLS_TEST2
from utils import make_lags

pd.set_option('display.max_columns', 100)

%matplotlib inline

In [2]:
configs = json.load(open('../configs/204_lgbm_custom_cv.json'))

In [3]:
# load feathers
files = sorted(glob('../feats/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)
df = df[configs['features']]
feats = [f for f in df.columns if f not in FEATS_EXCLUDED]

  0%|          | 0/113 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
100%|██████████| 113/113 [00:09<00:00, 11.80it/s]


In [5]:
# load model
reg = lgb.Booster(model_file='../output/lgbm_all_data.txt')

In [20]:
# Recursive prediction
for day in tqdm_notebook(range(1914,1914+28)):
    mask_test = (df['d_numeric']>=day-365)&(df['d_numeric']<=day)
    test_df = df[mask_test]
#        test_df = make_lags(test_df)
    df.loc[df['d_numeric']==day,'demand']=reg.predict(test_df[test_df['d_numeric']==day][feats], num_iteration=reg.best_iteration)

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [21]:
# split test
test_df = df[df['date']>='2016-04-25']

In [27]:
preds = test_df[['id','d','demand']].reset_index()
preds = preds.pivot(index='id', columns='d', values='demand').reset_index()

# split test1 / test2
preds1 = preds[['id']+COLS_TEST1]
preds2 = preds[['id']+COLS_TEST2]

# change column names
preds1.columns = ['id'] + ['F' + str(d + 1) for d in range(28)]
preds2.columns = ['id'] + ['F' + str(d + 1) for d in range(28)]

# replace test2 id
preds2['id']= preds2['id'].str.replace('_validation','_evaluation')

# merge
preds = preds1.append(preds2)

# save csv
preds.to_csv(submission_file_name, index=False)


/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


NameError: name 'submission_file_name' is not defined

In [9]:
test_df = df[(df['d_numeric']>=1914-365)&(df['d_numeric']<=1914)]

In [28]:
preds

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.781767,0.706037,0.685808,0.671023,0.893083,1.026479,1.089903,0.929458,0.845850,0.944168,0.870859,0.956588,1.160221,1.200294,0.976075,0.800041,0.776631,0.824477,0.915691,1.252093,1.123682,0.963222,0.839551,0.816733,0.763173,0.913984,1.141753,1.121180
1,FOODS_1_001_CA_2_validation,0.801976,0.737975,0.726140,0.719959,0.825413,0.914071,0.856236,0.697928,0.709968,0.744434,0.685616,0.802835,0.953470,0.843622,0.700140,0.626191,0.700814,0.622356,0.763707,0.866472,1.391542,1.008628,0.910871,0.877670,1.019525,1.124312,1.285415,1.363787
2,FOODS_1_001_CA_3_validation,1.216968,1.045000,1.049573,0.950526,0.953842,1.110647,1.154767,0.915415,0.890347,0.866152,0.807937,0.864334,1.276754,1.263695,1.136772,1.007610,0.940292,0.923882,0.969878,1.094779,1.757357,1.193331,1.017596,0.994689,0.990115,1.054564,1.339289,1.160244
3,FOODS_1_001_CA_4_validation,0.412360,0.386919,0.346325,0.346918,0.377182,0.422613,0.442406,0.382687,0.366617,0.351468,0.346554,0.362610,0.419517,0.368172,0.306907,0.285015,0.251829,0.252723,0.272145,0.328148,0.313327,0.314761,0.294756,0.272062,0.271135,0.321848,0.431385,0.458652
4,FOODS_1_001_TX_1_validation,0.496347,0.473144,0.431499,0.431008,0.432792,0.495436,0.553837,0.444915,0.472793,0.419698,0.418550,0.460972,0.508953,0.473623,0.441629,0.344994,0.353429,0.345955,0.365313,0.421538,0.426120,0.347130,0.306596,0.293866,0.289065,0.304502,0.359325,0.360700
5,FOODS_1_001_TX_2_validation,0.534945,0.498058,0.480484,0.552636,0.543715,0.543303,0.570731,0.454615,0.454700,0.425312,0.308909,0.457934,0.505516,0.536552,0.477901,0.437645,0.448721,0.503125,0.662712,0.613346,0.653930,0.526364,0.480364,0.469655,0.444620,0.426292,0.567346,0.489035
6,FOODS_1_001_TX_3_validation,0.410448,0.384957,0.380714,0.342425,0.437327,0.575228,0.634986,0.464391,0.514172,0.457541,0.454142,0.574986,0.556698,0.568679,0.520117,0.440728,0.449367,0.485494,0.480115,0.637006,0.646411,0.496087,0.449493,0.442012,0.505319,0.528022,0.601411,0.591098
7,FOODS_1_001_WI_1_validation,0.433679,0.432009,0.409862,0.430916,0.509459,0.547971,0.561141,0.397652,0.395960,0.368094,0.273481,0.428831,0.465021,0.405665,0.396449,0.361889,0.367763,0.368018,0.397994,0.475843,0.522733,0.376870,0.389472,0.381805,0.374896,0.434721,0.766778,0.733301
8,FOODS_1_001_WI_2_validation,0.294121,0.283977,0.271685,0.272513,0.274000,0.290127,0.471635,0.365371,0.364800,0.343777,0.370672,0.415578,0.446765,0.412574,0.418546,0.352656,0.482801,0.455285,0.439773,0.525207,0.525605,0.346771,0.363950,0.368552,0.350797,0.397807,0.456455,0.433101
9,FOODS_1_001_WI_3_validation,0.203499,0.204781,0.213356,0.219375,0.232967,0.220070,0.239776,0.215204,0.225170,0.214510,0.248634,0.279411,0.273180,0.257256,0.282239,0.241674,0.245530,0.240519,0.236741,0.299642,0.290751,0.228123,0.214407,0.211556,0.208915,0.226238,0.311695,0.304969
